# Lesion network mapping score computation

This script combines individual-level WMH masks with ROI-level functional and structural connectivity maps (computed via LEAD DBS) to obtain regional functional and structural lesion network mapping scores

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
code_dir=Path.cwd()
project_dir=code_dir.parent
input_dir=project_dir/"input"
output_dir=project_dir/"output/roiwise_lnm_scores"
tmp_dir=project_dir/"tmp"

output_dir.mkdir(exist_ok=True)

In [3]:
nifti_dirs = ["fLNM_GSP1000_schaefer400_plus_subcort", "fLNM_GSP1000_schaefer200","fLNM_GSP1000_schaefer100", 'fLNM_GSP1000_HCP842', 'fLNM_GSP1000_tian', 'sLNM_HCP32_schaefer400_plus_subcort','sLNM_HCP32_schaefer200','sLNM_HCP32_schaefer100', 'sLNM_HCP32_HCP842', 'sLNM_HCP32_tian']


# Prepare 4D niftis

In [4]:
schaefer400_labels = pd.read_csv(input_dir/"atlases/Schaefer400Parcels7Networks_labels.txt", delim_whitespace=True).columns.to_list()

hcp1065_labels = pd.read_csv(input_dir/"atlases/HCP1065_labels.txt", delim_whitespace=True, header=None, index_col=0).index.to_list()
hcp1065_labels = [i.replace(".nii.gz","") for i in hcp1065_labels]

tian_labels = pd.read_csv(input_dir/"atlases/Tian_Subcortex_S1_3T_label.txt", delim_whitespace=True, header=None, index_col=0).index.to_list()

roi_labels = schaefer400_labels + hcp1065_labels + tian_labels

In [7]:
# produce 4D LNM image by concatenating ROI_level functional / structural connectivity maps computed via "connectome_mapper" of LEAD DBS (4d: x,y,z,ROIs)
# mrcat $(cat sLNM_roi_paths.txt) sLNM_rois.nii -axis 3
# mrcat $(cat fLNM_roi_paths.txt) fLNM_rois.nii -axis 3

# Extract ROIwise LNM scores

In [ ]:
from joblib import Parallel, delayed
import pandas as pd
import nibabel as nib
from nilearn.image import resample_to_img

def process_LNM(WMH_img, WMH_path, LNM_type, roi, roi_idx, LNM_img_4d, LNM_img_4d_data, LNM_df):
    sub_name = WMH_path.name.split("_WMH")[0]
    if LNM_type == "sLNM": normative_connectome = "HCP32"; 
    elif LNM_type == "fLNM": normative_connectome = "GSP1000"
    if sub_name in LNM_df.index and f"{LNM_type}_{normative_connectome}_{roi}_pos_mean" in LNM_df.columns:
        if not LNM_df[f"{LNM_type}_{normative_connectome}_{roi}_pos_mean"].isna()[sub_name]:
            print(f"{sub_name} {LNM_type}_{normative_connectome}_{roi} already available: {LNM_df[f'{LNM_type}_{normative_connectome}_{roi}_pos'][sub_name]}")
            return
    else: print(f"Sampling {LNM_type} {normative_connectome} {roi} for {sub_name}")


    resampled_WMH_img = resample_to_img(WMH_img, LNM_img_4d, interpolation='nearest')
    WMH_data = resampled_WMH_img.get_fdata()
    LNM_data = LNM_img_4d_data[:,:,:,roi_idx]

    LNM_data_pos = LNM_data.copy()
    LNM_data_pos[LNM_data_pos < 0] = 0

    LNM_data_neg = LNM_data.copy()
    LNM_data_neg[LNM_data_neg > 0] = 0

    sum_ = np.nansum(LNM_data[WMH_data == 1])
    mean_ = np.nanmean(LNM_data[WMH_data == 1])

    pos_sum = np.nansum(LNM_data_pos[WMH_data == 1])
    pos_mean = np.nanmean(LNM_data_pos[WMH_data == 1])

    neg_sum = np.nansum(LNM_data_neg[WMH_data == 1])
    neg_mean = np.nanmean(LNM_data_neg[WMH_data == 1])

    LNM_data_pos_p75 = np.nanpercentile(LNM_data_pos[WMH_data == 1], 75)
    LNM_data_pos_p50 = np.nanpercentile(LNM_data_pos[WMH_data == 1], 50)

    LNM_data_neg_p25 = np.nanpercentile(LNM_data_neg[WMH_data == 1], 25)
    LNM_data_neg_p50 = np.nanpercentile(LNM_data_neg[WMH_data == 1], 50)

    LNM_data_pos[LNM_data_pos < LNM_data_pos_p50] = None
    pos_mean_p50 = np.nanmean(LNM_data_pos[WMH_data == 1])

    LNM_data_pos[LNM_data_pos < LNM_data_pos_p75] = None
    pos_mean_p75 = np.nanmean(LNM_data_pos[WMH_data == 1])

    LNM_data_neg[LNM_data_neg > LNM_data_neg_p50] = None
    neg_mean_p50 = np.nanmean(LNM_data_neg[WMH_data == 1])

    LNM_data_neg[LNM_data_neg > LNM_data_neg_p25] = None
    neg_mean_p25 = np.nanmean(LNM_data_neg[WMH_data == 1])

    return pd.DataFrame({f"{LNM_type}_{normative_connectome}_{roi}_sum": sum_,
                         f"{LNM_type}_{normative_connectome}_{roi}_mean": mean_,
                         f"{LNM_type}_{normative_connectome}_{roi}_pos_sum": pos_sum, 
                         f"{LNM_type}_{normative_connectome}_{roi}_pos_mean": pos_mean, 
                         f"{LNM_type}_{normative_connectome}_{roi}_pos_mean_peak50": pos_mean_p50,
                         f"{LNM_type}_{normative_connectome}_{roi}_pos_mean_peak25": pos_mean_p75,
                         f"{LNM_type}_{normative_connectome}_{roi}_neg_sum": neg_sum,
                         f"{LNM_type}_{normative_connectome}_{roi}_neg_mean": neg_mean,
                         f"{LNM_type}_{normative_connectome}_{roi}_neg_mean_peak50": neg_mean_p50,
                         f"{LNM_type}_{normative_connectome}_{roi}_neg_mean_peak25": neg_mean_p25
                         }, dtype=float, index=[sub_name])

# Process in parallel
def process_all_WMH_images(LNM_type, LNM_df, roi_labels):
    
    LNM_img_4d = nib.load(project_dir/f"output/atlas_rois/{LNM_type}_rois.nii")
    
    for idx, WMH_path in enumerate(WMH_paths):
        WMH_img = nib.load(WMH_path)
        LNM_img_4d_data = LNM_img_4d.get_fdata()
        assert LNM_img_4d_data.shape[3] == len(roi_labels), "4th dimension does not equal roi_labels lengths"
        LNM_dfs = Parallel(n_jobs=15)(delayed(process_LNM)(WMH_img, WMH_path, LNM_type, roi, roi_idx, 
                        LNM_img_4d, LNM_img_4d_data, LNM_df) for roi_idx, roi in enumerate(roi_labels))
        # Aggregate results in memory
        if not all(element is None for element in LNM_dfs):
            
            temp_df = pd.concat(LNM_dfs, axis=1, ignore_index=False)
            LNM_df = LNM_df.combine_first(temp_df)
        else: continue

        if idx % 100 == 0:
            print(idx)
        if idx % 500 == 0:
            LNM_df.to_csv(output_dir/"LNM_df.csv")

    return temp_df, LNM_df

# Pre-compute nifti_paths before the loop
LNM_types = ["fLNM","sLNM"]
roi_labels = list(roi_labels_df.values.flatten())

# Load LNM_df once
LNM_df = pd.DataFrame()
WMH_dir = input_dir/"WMH_nii"
WMH_paths = sorted(WMH_dir.glob("*"), reverse=True)
WMH_paths.reverse()

# Loop over LNM_types and process images
for LNM_type in LNM_types:
    temp_df, LNM_df = process_all_WMH_images(LNM_type, LNM_df, roi_labels)
    LNM_df.to_csv(output_dir/f"{LNM_type}_LNM_df.csv")
